In [1]:
import numpy as np
np.set_printoptions(suppress=True)

In [2]:
from tpk4170.visualization import Viewer
from tpk4170.models import Grid, Axes, PointCloud
from tpk4170.rigid_body_motions.rotations import vec, exp

# Procrustes

We want to find the rotation matrix $\mathbf{R} \in SO(3)$ that best fits $\mathbf{a}_i = \mathbf{Rb}_i,\ i=1,2,\ldots, n$, by solving the minimization problem 
\begin{align}
\min_{\mathbf{R}\in SO(3)} f 
\end{align}
where 
\begin{align}
f = \sum_{i=1}^n \lVert \mathbf{Ra}_{i} - \mathbf{b}_i  \rVert^2 = \lVert \mathbf{RA} - \mathbf{B} \rVert_F^2
\end{align}
and
\begin{align}
\mathbf{A} = (\mathbf{a}_1, \mathbf{a}_2, \ldots, \mathbf{a}_n) \in \mathbb{R}^{3\times n}
\quad\text{and}\quad 
\mathbf{B} = (\mathbf{b}_1, \mathbf{b}_2, \ldots, \mathbf{b}_n) \in \mathbb{R}^{3\times n}
\end{align}

## Load model

In [3]:
A = np.loadtxt('./data/bunny.txt').T * 10.
print(A[:,:4])

[[-0.378297  -0.447794  -0.680095  -0.0228741]
 [ 1.2794     1.28887    1.51244    1.3015   ]
 [ 0.0447467  0.0190497  0.371953   0.232201 ]]


## Create rotation matrix

In [4]:
theta = np.pi/3
k = vec(1,0,0)
R = exp(theta * k)
print(R)

[[ 1.         0.         0.       ]
 [ 0.         0.5       -0.8660254]
 [ 0.         0.8660254  0.5      ]]


## Apply rotation

In [5]:
B = R @ A
print(B[:,:4])

[[-0.378297   -0.447794   -0.680095   -0.0228741 ]
 [ 0.60094822  0.62793748  0.43409925  0.44965804]
 [ 1.13036625  1.12571901  1.49578796  1.24323256]]


## Visualize

In [6]:
viewer = Viewer()
viewer.add(Grid())
viewer.add(Axes())

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(color='white', intensity=0.66, positi…

In [7]:
viewer.add(PointCloud(B.T, color='red'))
viewer.add(PointCloud(A.T))

## Solve for rotation

In [8]:
def solve_for_rotation(A, B):
    H = np.dot(A, B.T)
    U, S, Vt = np.linalg.svd(H)
    R = np.dot(Vt.T, U.T)
    if np.linalg.det(R) < 0:
        Vt[2, :] *= -1
        R = np.dot(Vt.T, U.T)
    return R

In [9]:
Re = solve_for_rotation(A,B)
print(Re)

[[ 1.        -0.        -0.       ]
 [-0.         0.5       -0.8660254]
 [ 0.         0.8660254  0.5      ]]


In [10]:
print(R)

[[ 1.         0.         0.       ]
 [ 0.         0.5       -0.8660254]
 [ 0.         0.8660254  0.5      ]]


In [11]:
print(np.allclose(R,Re))

True
